<font color='blue'><H1><B><font size="14"><center> Segmenting and Clustering Neighborhoods in Toronto</H1></B></center></font>

<font><H1><B> Week 3 Assignment - IBM Capstone project</H1><B></font>

### *Step 0: Installing and Importing the needed libraries*

In [142]:
!pip install BeautifulSoup4
!pip install requests
import random # library for random number generation
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library to process data as dataframes/for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline 

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

!pip install folium
import folium # map rendering library
from html.parser import HTMLParser
import requests
from bs4 import BeautifulSoup
import urllib
import urllib.request
import time

!pip install geocoder
import geocoder # import geocoder

from IPython.display import clear_output

print('Libraries imported.')

Libraries imported.


In [143]:
try:
    import pgeocode
    print ('Postal Geocoder available')
except ImportError:
    # installing geocoder
    print ('Installing Postal Geocoder')
    !pip install geocoder
    clear_output()
    print ('Postal Geocoder Installed')
    
import geocoder # import geocoder

Postal Geocoder Installed


## **Part 1: Scrap the wikipedia table containing the postal codes, boroughs and neighborhood of Toronto, create a dataframe using pandas and clean the fetched data**
***

### *Part 1.1: Getting the Data from wikipedia tables*

In [144]:
# Web Scraping Wikipedia Tables 
website_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(website_url)
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")
#print(soup.prettify())

#finding the table
MyTorontoTable= soup.find("table",{"class":"wikitable sortable"})
#print(soup.title.string)  checking the extracted info by checking the title of the table
#MyTorontoTable - checking the table

### *Part 1.2: Reading the data scraped from the table into an array* 

In [145]:
#There are three columns in our table that we want to scrape the data from so we will set up three empty lists to store our data in.
wiki_postalcode=[]
wiki_borough=[]
wiki_neighbourhood=[]
columns = []

for row in MyTorontoTable.findAll('tr'):
    # section = []
   # for td in tr.find_all(['th','td']):
       # section.append(td.text.rstrip())
    cells=row.findAll('td')
    if len(cells) > 0 :
        wiki_postalcode.append(cells[0].text.strip())
        wiki_borough.append(cells[1].text.strip())
        wiki_neighbourhood.append(cells[2].text.strip())
        

### *Part 1.3: Creating a Dataframe and adding data to it*

In [146]:
# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood'] 
TorontoDF=pd.DataFrame(columns=column_names)
TorontoDF['PostalCode']= wiki_postalcode
TorontoDF['Borough']= wiki_borough
TorontoDF['Neighborhood']= wiki_neighbourhood
TorontoDF.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### *Part 1.4: Printing the size of the Dataframe before cleaning data as requested*

In [147]:
TorontoDF.shape

(288, 3)

### *Part 1.5: Cleaning the Data in the Dataframe (as per the criteria requested in the assignment*

#### 1.5.1 Cleaning the data: read into the dataframe (remove Boroughs containing "Not assigned")

In [148]:
#Preparing Data in the Data frame
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
TorontoDF.drop(TorontoDF[TorontoDF['Borough']=="Not assigned"].index,axis=0, inplace=True)
#TorontoDF.head(10)

#### 1.5.2 Cleaning the data: read into the dataframe (Neighborhood to be assigned the same as Borough in case Borough has a neighborhood not assigned)

In [149]:
#Preparing Data in the Data frame
## If a cell has a borough but a Not assigned neighborhood, # then the neighborhood will be the same as the borough. 
# So for the 9th cell in the table on the Wikipedia page, # the value of the Borough and the Neighborhood columns will be Queen's Park.
TorontoDF['Neighborhood'].replace("Not assigned", TorontoDF["Borough"],inplace=True)
#TorontoDF.head(10)

#### 1.5.3 Cleaning the data: read into the dataframe (More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma)

In [150]:
#Preparing Data in the Data frame
# More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma
TorontoDF_xy = TorontoDF.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
TorontoDF= TorontoDF_xy.copy()
#TorontoDF.head(10)

### *Part 1.6: Displaying the Dataframe to ensure that Data is prepared and filtered as requested*

In [151]:
TorontoDF.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### *Part 1.7: Printing the size of the Dataframe after cleaning and filtering*

In [152]:
TorontoDF.shape

(103, 3)

## **Part 2: Getting the coordinates of each postal code and adding it to the Dataframe**
***

### *Part 2.1: Reading the coordinates from .csv file*

In [153]:
# Step : read the coordinates from a csv file
csv_file_url = "http://cocl.us/Geospatial_data "
neigh_longlat_df=pd.read_csv(csv_file_url)
print ("file read into DataFrame")


file read into DataFrame


### *Part 2.2: Adding each latitide and Longitide to the corresponding Postal Code in the current Data frame*

In [154]:
neigh_longlat_df.head()
neigh_longlat_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
neigh_longlat_df.head()
TorontoDF= pd.merge(TorontoDF, neigh_longlat_df[['PostalCode', 'Latitude', 'Longitude']], on='PostalCode')
TorontoDF.head(11)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### *Part 2.3: Just a checkup point (to confirm the size of the Dataframe) before starting analysis of Neighborhoods (part 3)*

In [155]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(TorontoDF['Borough'].unique()),
        TorontoDF.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


### *Part 2.4: Create a map of Toronto with neighborhoods superimposed on top (as a warm-up for part 3)*

In [156]:
address = 'Toronto, CANADA'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [157]:
# create map of Toronto using latitude and longitude values
import folium # map rendering library
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(TorontoDF['Latitude'], TorontoDF['Longitude'], TorontoDF['Borough'], TorontoDF['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

# **Part 3: Explore and cluster the neighborhoods in Toronto**
***

## *Part 3.1: Passing Foursquare Credetentials*

In [181]:
CLIENT_ID = 'UUK00KDCJYXQ34USIX2WY1E1S2BW1FGPKNOSXOWZ5MXYEQRJ' # your Foursquare ID
CLIENT_SECRET = 'LHGFO5RD3TJW3S4ZSV2LYLSBYEWHOH0NX4EPLHUURCOXWR2C' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

## *Part 3.2: Selecting Neighborhoods inwhich the Borough includes word Toronto as part of its name & add it to a temporary Dataframe*

In [182]:
boroughs_includes_toronto=TorontoDF[TorontoDF['Borough'].str.contains('Toronto')]
boroughs_includes_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


### Part 3.3.1: Printing the size of the Temporary Datframe inwhich the Borough includes word Toronto as part of its name

In [183]:
print('There are {} ' .format(boroughs_includes_toronto.shape[0]) + 'Neighborhoods where the Borough includes the word Toronto')

There are 38 Neighborhoods where the Borough includes the word Toronto


## *Part 3.3: Exploring the First Neighborhood in our Dataframe (for boroughs including the word Toronto)*

### Part 3.3.1: Getting the index of the first neighborhood in the temporary Dataframe

In [184]:
index_of_1st_neighborhood= boroughs_includes_toronto.index[0]
index_of_1st_neighborhood

37

### Part 3.3.2: Getting Neighborhood latitude and longitude

In [185]:
neighborhood_latitude = boroughs_includes_toronto.loc[index_of_1st_neighborhood, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = boroughs_includes_toronto.loc[index_of_1st_neighborhood, 'Longitude'] # neighborhood longitude value

neighborhood_name = boroughs_includes_toronto.loc[index_of_1st_neighborhood, 'Neighborhood'] # neighborhood name
#neighborhood_borough = boroughs_includes_toronto.loc[index_of_1st_neighborhood, 'Borough'] # neighborhood name
#print(neighborhood_borough)

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


### Part 3.3.3: Getting 100 venues in the neighborhood within radius 500

In [186]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=UUK00KDCJYXQ34USIX2WY1E1S2BW1FGPKNOSXOWZ5MXYEQRJ&client_secret=LHGFO5RD3TJW3S4ZSV2LYLSBYEWHOH0NX4EPLHUURCOXWR2C&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [187]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d2c96868ad62e002cd46825'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

### Part 3.3.4: Extracting the categories of the venues

In [188]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [189]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
4,Upper Beaches,Neighborhood,43.680563,-79.292869


### Part 3.3.5: Getting nearby venues

In [190]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    boroughs_includes_toronto = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    boroughs_includes_toronto.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(boroughs_includes_toronto)

In [191]:
Toronto_venues = getNearbyVenues(names=boroughs_includes_toronto['Neighborhood'],
                                   latitudes=boroughs_includes_toronto['Latitude'],
                                   longitudes=boroughs_includes_toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [192]:
Toronto_venues.groupby('Neighborhood').count()
Toronto_venues.head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
5,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
6,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
7,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
8,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
9,"The Danforth West, Riverdale",43.679557,-79.352188,La Diperie,43.677530,-79.352295,Ice Cream Shop


### Part 3.3.6: Analyzing each Neighborhood

In [193]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

#Toronto_onehot.head(20)

In [194]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,0.00,0.010000,0.010000,0.000000,0.030000,0.0000,0.000000,0.000000,0.0000,0.000000,0.030000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.010000,0.00000,0.010000,0.020000,0.000000,0.000000,0.00,0.020000,0.010000,0.00,0.000000,0.050000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.070000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0

In [195]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2        Bakery  0.05
3    Steakhouse  0.04
4          Café  0.04


----Brockton, Exhibition Place, Parkdale Village----
                   venue  freq
0            Yoga Studio  0.07
1         Breakfast Spot  0.07
2                   Café  0.07
3            Coffee Shop  0.07
4  Performing Arts Venue  0.04


----Business Reply Mail Processing Centre 969 Eastern----
           venue  freq
0    Yoga Studio  0.05
1     Restaurant  0.05
2     Skate Park  0.05
3    Pizza Place  0.05
4  Burrito Place  0.05


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.19
1  Airport Terminal  0.12
2    Airport Lounge  0.12
3  

In [196]:
Toronto_grouped.shape

(38, 239)

In [197]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [198]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,American Restaurant,Thai Restaurant,Gym,Sushi Restaurant,Bakery,Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Beer Bar,Steakhouse,Cheese Shop,Café,Seafood Restaurant,Basketball Stadium
2,"Brockton, Exhibition Place, Parkdale Village",Yoga Studio,Breakfast Spot,Café,Coffee Shop,Bakery,Stadium,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Smoke Shop,Skate Park,Brewery,Burrito Place,Butcher,Restaurant,Recording Studio,Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Bar,Boutique,Airport,Airport Food Court,Airport Gate,Boat or Ferry,Harbor / Marina
5,"Cabbagetown, St. James Town",Coffee Shop,Bakery,Café,Pizza Place,Restaurant,Market,Italian Restaurant,Pet Store,Pub,Bank
6,Central Bay Street,Coffee Shop,Café,Ice Cream Shop,Italian Restaurant,Chinese Restaurant,Burger Joint,Sandwich Place,Salad Place,Indian Restaurant,Spa
7,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Bar,Mexican Restaurant,Bakery,Chinese Restaurant,Dumpling Restaurant,Coffee Shop,Comfort Food Restaurant
8,Christie,Grocery Store,Café,Park,Baby Store,Convenience Store,Italian Restaurant,Coffee Shop,Athletics & Sports,Diner,Restaurant
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym


### Part 3.3.7: Cluster the Neighborhoods

In [199]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [200]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'TORONTO_Clusters', kmeans.labels_)

Toronto_merged = boroughs_includes_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,TORONTO_Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Other Great Outdoors,Trail,Pub,Wings Joint,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Liquor Store,Bakery,Sports Bar,Spa,Juice Bar
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Park,Sandwich Place,Board Shop,Pub,Liquor Store,Burger Joint,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse
43,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Italian Restaurant,Gastropub,American Restaurant,Bakery,Yoga Studio,Bar,Fish Market,Coworking Space
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Gym / Fitness Center,Park,Swim School,Bus Line,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


### Part 3.3.8: Creating a map with the clusters 

In [202]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['TORONTO_Clusters']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Part 3.3.9: Examining the clusters

In [203]:
Toronto_merged.loc[Toronto_merged['TORONTO_Clusters'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,TORONTO_Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,0,Playground,Restaurant,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [204]:
Toronto_merged.loc[Toronto_merged['TORONTO_Clusters'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,TORONTO_Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,East Toronto,1,Park,Sandwich Place,Board Shop,Pub,Liquor Store,Burger Joint,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse
45,Central Toronto,1,Gym,Clothing Store,Breakfast Spot,Food & Drink Shop,Sandwich Place,Hotel,Asian Restaurant,Park,Grocery Store,Pizza Place
50,Downtown Toronto,1,Park,Playground,Trail,Building,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
64,Central Toronto,1,Bus Line,Sushi Restaurant,Park,Jewelry Store,Trail,Doner Restaurant,Discount Store,Dive Bar,Dog Run,Wings Joint


In [205]:
Toronto_merged.loc[Toronto_merged['TORONTO_Clusters'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,TORONTO_Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,2,Health Food Store,Other Great Outdoors,Trail,Pub,Wings Joint,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant
41,East Toronto,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Liquor Store,Bakery,Sports Bar,Spa,Juice Bar
43,East Toronto,2,Café,Coffee Shop,Italian Restaurant,Gastropub,American Restaurant,Bakery,Yoga Studio,Bar,Fish Market,Coworking Space
46,Central Toronto,2,Clothing Store,Coffee Shop,Health & Beauty Service,Sporting Goods Shop,Chinese Restaurant,Dessert Shop,Diner,Fast Food Restaurant,Italian Restaurant,Mexican Restaurant
47,Central Toronto,2,Dessert Shop,Sandwich Place,Pizza Place,Italian Restaurant,Sushi Restaurant,Coffee Shop,Café,Gourmet Shop,Farmers Market,Restaurant
49,Central Toronto,2,Coffee Shop,Pub,Pizza Place,American Restaurant,Light Rail Station,Sports Bar,Bagel Shop,Restaurant,Supermarket,Sushi Restaurant
51,Downtown Toronto,2,Coffee Shop,Bakery,Café,Pizza Place,Restaurant,Market,Italian Restaurant,Pet Store,Pub,Bank
52,Downtown Toronto,2,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym
53,Downtown Toronto,2,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Mexican Restaurant,Theater,Restaurant,Gym / Fitness Center,Café
54,Downtown Toronto,2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Diner,Plaza,Pizza Place,Ramen Restaurant


In [206]:
Toronto_merged.loc[Toronto_merged['TORONTO_Clusters'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,TORONTO_Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,3,Gym / Fitness Center,Park,Swim School,Bus Line,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [207]:
Toronto_merged.loc[Toronto_merged['TORONTO_Clusters'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,TORONTO_Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,4,Garden,Home Service,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


# End of Assignment 